# Importing Packages

In [ ]:
import json
import os

In [ ]:
from qiime2.plugins import feature_table
from qiime2 import Artifact
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import biom
import seaborn as sns

## Importing Data

In [ ]:
## Obtaining file paths
with open("config/data-params.json") as fh:
    file_paths = json.load(fh)

In [ ]:
feature_table = Artifact.load(file_paths["feature_table_path"])
metadata = pd.read_csv(file_paths["metadata_path"], sep='\t', index_col=0)

In [ ]:
biom_table = feature_table.view(biom.Table)
print(biom_table.head())

In [ ]:
metadata.head()

## Missing Data Analysis

In [ ]:
# Function for single representation for missing values
def missing_values(col, type='categorical'):
    temp = col.apply(lambda x: np.nan if x == 'not applicable' or x == 'not provided' else x)
    if type == 'numeric':
        temp = temp.apply(lambda x: x if pd.isnull(x) else np.float64(x))
    return temp

In [ ]:
# Potential Columns -- more to be added 
features = ['abdominal_obesity_ncep_v2', 'ckd_v2','diabetes2_v2','precvd_v2', 'elevated_bp_selfmeds_v2', 'high_total_chol2_v2',
'gender_v2','agegroup_c6_v2','bmigrp_c6_v2','center','us_born_v2','income_c5_v2'] # 'host_age'

In [ ]:
metadata['abdominal_obesity_ncep_v2'].value_counts()

In [ ]:
metadata['ckd_v2'].value_counts()

In [ ]:
missing_values(metadata['ckd_v2'],'numeric').value_counts()

# Subset of Metadata

In [ ]:
sub_metadata = metadata[features]
sub_metadata.head()

In [ ]:
numeric_col = ['abdominal_obesity_ncep_v2','ckd_v2','diabetes2_v2','precvd_v2','elevated_bp_selfmeds_v2','high_total_chol2_v2','agegroup_c6_v2','bmigrp_c6_v2','us_born_v2','income_c5_v2']
categorical_col = ['center','gender_v2']

## Defining missing values

In [ ]:
sub_metadata.loc[:,numeric_col] = sub_metadata[numeric_col].apply(lambda x: missing_values(x,'numeric'))
sub_metadata.loc[:,categorical_col] = sub_metadata[categorical_col].apply(lambda x: missing_values(x,'categorical'))

In [ ]:
sub_metadata.isnull().sum() * 100 / len(sub_metadata) # calculates percent of nan in column


## EDA on Subset of Metadata

In [ ]:
metadata['ckd_v2'].value_counts()

In [ ]:
sub_metadata['ckd_v2'].value_counts()

In [ ]:
sub_metadata.describe()

### Disease Counts

In [ ]:
def create_bar_col_binary(df, col_name):
    ax = df[col_name].fillna(-1).value_counts().sort_index(ascending=False).plot(kind='barh')
    ax.set_xlabel('count')
    ax.set_ylabel('outcome')
    ax.set_title(col_name)

In [ ]:
create_bar_col_binary(sub_metadata, 'ckd_v2')

### Gender Counts

In [ ]:
sub_metadata['gender_v2'].fillna('missing').value_counts().plot(kind='barh')

### Center Counts

In [ ]:
sub_metadata['center'].fillna('missing').value_counts().plot(kind='barh')

### Co-occurence

In [ ]:
sub_metadata

In [ ]:
sub_metadata_no_nan = sub_metadata.dropna()
sub_metadata_no_nan.shape[0]/sub_metadata.shape[0]

In [ ]:
convert_dict = {x:np.float64 for x in numeric_col}

In [ ]:
sub_metadata_no_nan = sub_metadata_no_nan.astype(convert_dict)

In [ ]:
disease_col = ['abdominal_obesity_ncep_v2','ckd_v2','diabetes2_v2','precvd_v2','elevated_bp_selfmeds_v2','high_total_chol2_v2']

In [ ]:
sub_metadata[['ckd_v2','abdominal_obesity_ncep_v2']].dropna().T.dot(sub_metadata[['ckd_v2','abdominal_obesity_ncep_v2']].dropna())

In [ ]:
sns.heatmap(sub_metadata_no_nan[disease_col].T.dot(sub_metadata_no_nan[disease_col]))

# Model Creation

In [ ]:
def binary_to_tf(val):
    if val==1.0:
        return 'T'
    elif val==0.0:
        return 'F'
    else:
        return 'missing'

In [ ]:
sub_metadata['abdominal_obesity_ncep_v2'] = sub_metadata['abdominal_obesity_ncep_v2'].apply(lambda x: binary_to_tf(x))

In [ ]:
sub_metadata.to_csv("data/temp/updated_metadata.tsv",sep="\t")

convert df to qiime metadata

In [ ]:
import os

### Qiime metadata

Only need the following if using Artifact API

In [ ]:
from qiime2 import Metadata

In [ ]:
qiime_metadata = Metadata.load("data/temp/updated_metadata.tsv")
qiime_metadata.save('qiime_metadata.qza')